Because the simulation workflow outlined in [GSSHA_Workflow](GSSHA_Workflow.ipynb) is fully scripted, supporting but not requiring any user interaction, it's straightforward to run parameter sweeps and other automated tasks.  Here we demonstrate running simulations covering a range of rain intensities, to show how to collect and interpret data across a range of parameter values.

In [ ]:
from datetime import timedelta
import os
import glob

import numpy as np
import xarray as xr
import geoviews as gv
import holoviews as hv
import cartopy.crs as ccrs

from gsshapy.modeling import GSSHAFramework

import earthsim.gssha as eagssha
import earthsim.gssha.model as models
from earthsim.gssha import get_file_from_quest
from earthsim.io import open_gssha, get_ccrs

hv.extension('bokeh')

## Set up simulation

In [ ]:
def get_simulation(**params):
    name = 'vicksburg_parameterized_%s' % '_'.join(['%s:%s' % item for item in sorted(params.items())])
    data_dir = 'vicksburg_south'
    model_creator = eagssha.CreateGSSHAModel(name='Vicksburg South Model Creator',
                                            mask_shapefile='../data/vicksburg_watershed/watershed_boundary.shp',
                                            grid_cell_size=90, project_name=name)
    sim = eagssha.Simulation(name='Vicksburg South Simulation', simulation_duration=60*60,
                              rain_duration=30*60, model_creator=model_creator, **params)
    sim.model_creator.roughness = models.GriddedRoughnessID(
        land_use_grid=get_file_from_quest(data_dir, sim.land_use_service, 'landuse', sim.model_creator.mask_shapefile),
        land_use_grid_id=sim.land_use_grid_id)
    sim.model_creator.elevation_grid_path = get_file_from_quest(data_dir, sim.elevation_service, 'elevation', sim.model_creator.mask_shapefile)

    model = sim.model_creator()
    model.project_manager.setCard('FLOOD_GRID',
                                  '{0}.fgd'.format(sim.model_creator.project_name),
                                  add_quotes=True)
    model.project_manager.setCard('DEPTH', '.', add_quotes=True)
    model.project_manager.setCard('MAP_FREQ', '1')
    model.write()
    return sim


def run_simulation(sim):
    project_path = os.path.join(sim.model_creator.project_base_directory, sim.model_creator.project_name)
    gr = GSSHAFramework("gssha",
        project_path,
        "{0}.prj".format(sim.model_creator.project_name),
        gssha_simulation_start=sim.simulation_start,
        gssha_simulation_duration=timedelta(seconds=sim.simulation_duration),
    )
    gr.event_manager.add_uniform_precip_event(sim.rain_intensity, 
                                              timedelta(seconds=sim.rain_duration))
    return convert_xarray(gr.run())


def convert_xarray(event_directory):
    depth_nc = os.path.join(event_directory, 'depths.nc')
    if not os.path.isfile(depth_nc):
        # Load depth data files
        depth_map = hv.HoloMap(kdims=['Minute'])
        for fname in glob.glob(os.path.join(event_directory, 'depth.*.asc')):
            depth_arr = open_gssha(fname)
            minute = int(fname.split('.')[-2])
            # NOTE: Due to precision issues not all empty cells match the NaN value properly, fix later
            depth_arr.data[depth_arr.data==depth_arr.data[0,0]] = np.NaN
            depth_map[minute] = hv.Image(depth_arr)

        # Convert data to an xarray and save as NetCDF
        arrays = []
        for minute, img in depth_map.items():
            ds = hv.Dataset(img)
            arr = ds.data.z.assign_coords(minute=minute)
            arrays.append(arr)
        depths = xr.concat(arrays, 'minute')
        depths.to_netcdf(depth_nc)
    else:
        depths = xr.open_dataset(depth_nc)
    return depths

## Set up models

In [ ]:
intensities = [128, 256, 512]
simulations = {intensity: get_simulation(rain_intensity=intensity) for intensity in intensities}

## Run models

In [ ]:
evaluated = {i: run_simulation(sim) for i, sim in simulations.items()}

## Load simulation data

In [ ]:
arrays = []
for i, xarr in sorted(evaluated.items()):
    arrays.append(xarr.assign_coords(intensity=i))
xarr = xr.concat(arrays, 'intensity')

In [ ]:
sim = simulations[512]
CRS = get_ccrs(os.path.join(sim.model_creator.project_name,
                            sim.model_creator.project_name+'_prj.pro'))

## Display simulation data

In [ ]:
%%opts Image [width=600 height=400 logz=True colorbar=True] (cmap='viridis')
ds = gv.Dataset(xarr, crs=CRS)
tiles = gv.WMTS('http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png', crs=ccrs.PlateCarree(),
                extents=(-91, 32.2, -90.8, 32.4))
tiles * ds.to(gv.Image, ['x', 'y'], dynamic=True).redim.range(z=(0, 1))

Here the user can explore plots across the various minute values and intensity settings to see how the results differ. In this example, all of the simulations were launched from a single notebook file, which makes it simple to explain how it works but in general one can use a parameterized notebook file and launch multiple independent runs, each filling out one notebook, perhaps on many nodes of an HPC system (e.g. using [Lancet](http://github.com/ioam/lancet)).  As the jobs complete, the output can be collated and then displayed for the user as above.